In [1]:
from copy import deepcopy
from colorama import Fore, Back, Style
'''
Với qui tắc là không được di chuyển theo đường chéo hay phương xiên
Biến DIRECTIONS là một mảng gồm bốn số nguyên đại diện cho 4 hướng di chuyển trên bài toán puzzle.
Số nguyên đầu tiên đại diện cho hàng và số nguyên thứ hai đại diện cho cột.
[-1,0]: hướng bắc (đi lên), [1,0]: hướng nam (đi xuống), [0,-1]: hướng tây (qua trái), [0,1]: hướng đông (qua phải)
'''
#Ma trận hướng đi
DIRECTIONS = {"U": [-1, 0], "D": [1, 0], "L": [0, -1], "R": [0, 1]}

'''
END là trạng thái đích của ma trận với các ô mà mình muốn trạng thái ban đầu trở thành.
'''
#Ma trận đích 
END = [[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,0]]

 
#Thanh màu
'''
tất cả các màu sẽ được đặt lại trở lại giá trị mặc định 
của chúng khi chương trình này chạy xong để chúng có thể bắt đầu lại 
nếu cần mà không có bất kỳ màu nào còn sót lại từ các lần chạy trước 
của chương trình này hoặc các chương trình khác sử dụng cùng màu này 
cho những thứ khác nhau như vẽ hình dạng hoặc điền vào các ô trên bảng tính
'''
bar = Style.BRIGHT + Fore.BLACK + '\u2502' + Fore.RESET + Style.RESET_ALL
#Thanh trắng dọc
dash='\u2500'
#Thanh trắng ngang
right_junction = '\u2524'
#Thanh trắng ngang
left_junction = '\u251C'

#hàm vẽ puzlle
'''
 Hàm bắt đầu bằng cách tạo một danh sách trống được gọi là array.
 Sau đó, nó lặp lại qua độ dài của danh sách, đó là len (array) trong trường hợp này.
 Sau đó, nó lặp qua từng phần tử của danh sách và in một thanh với Back.WHITE ở trên cùng, sau đó là một khoảng trắng và sau đó Back.RESET để chỉ ra rằng không còn phần tử nào trong danh sách.
Tiếp theo bắt đầu in các thanh cho mọi phần tử trong danh sách ngoại trừ 0, đó là lý do tại sao nó ghi "else" ở cuối dòng đó thay vì chỉ "if i == 0".
'''
def print_puzzle(array):
    for a in range(len(array)):
        for i in array[a]:
            if i == 0:
                print(bar, Back.WHITE + ' ' + Back.RESET, end=' ')
            else:
                print(bar, i, end=' ')
        print(bar)

        

#nó là nút lưu trữ từng trạng thái câu đố
'''
 Một định nghĩa lớp cho Node.
 Phương thức __init__ là hàm xây dựng tạo ra một thể hiện của lớp và nó có ba tham số: current_node, previous_node và g, h. 
 Phương pháp f trả về tổng của g và h.
 Mã sẽ tạo một nút.
 Nút được tạo với các biến current_node, previous_node và g, h.
 Hàm __init__ tạo đối tượng Node bằng cách khởi tạo các biến current_node, previous_node và g, h của nó.
'''
class Node:
    def __init__(self, current_node, previous_node, g, h, dir):
        self.current_node = current_node
        self.previous_node = previous_node
        self.g = g
        self.h = h
        self.dir = dir
        
    def f(self):
        return self.g + self.h

#trả về vị trí 
'''
Hàm này nhận vào một ma trận và phần tử e cần tìm
Trả về vị trí row, col của phần tử đó trong ma trận
Cách triển khai sẽ là duyệt từng dòng (row) trong ma trận
'''
def get_pos(current_state, element):
    for row in range(len(current_state)):
        if element in current_state[row]:
            return (row, current_state[row].index(element))
        
        
'''
Hàm tính khoảng cách giữa hai điểm trên lưới
Điểm đầu tiên nằm ở cuối danh sách và điểm thứ hai nằm ở hàng 0, cột 1
Duyệt qua từng dòng cho đến khi đạt đến len (current_state) - 1 (hàng cuối cùng).
Tiếp tục duyệt qua từng cột cho đến khi đạt len (current_state) - 1 (cột cuối cùng).
Đối với mọi vị trí trên lưới này, get_pos() trả về một số nguyên tương ứng với vị trí của nó so với END.
Tính chi phí khoảng cách dựa trên công thức  manhattan
'''
#Tính khoảng cách giữa các nút
def Distance(current_state):
    cost = 0
    for row in range(len(current_state)):
        for col in range(len(current_state[0])):
            pos = get_pos(END, current_state[row][col])
            cost += abs(row - pos[0]) + abs(col - pos[1])
    return cost


'''
Triển khai một danh sách listNode
Mã sau đó lặp lại thông qua từ điển trong DIRECTIONS và đối với mỗi khóa, nó tính toán vị trí mới của node.current_node dựa 
trên vị trí hiện tại của nó và hướng được truyền vào như một đối số để getAdjNode ().

Ví dụ: nếu nút ở mức (0, 0) và chúng tôi muốn di chuyển nó sang trái một bước, thì newPos sẽ là (0, -1).

Sau khi tính toán vị trí mới của node.current_node bằng phương pháp này cho tất cả các khóa trong
từ điển DIRECTIONS cho đến khi không còn khóa nào trong từ điển DIRECTIONS 
hoặc len (node.current_node) đã đạt được; hàm trả về một bản sao deepcopy() của 
những gì hiện đang được lưu trữ bên trong node.current_node với các vị trí trống chứa 
đầy số không để chúng không can thiệp vào tính toán của các node khác khi thêm chúng vào 
listNode sau này.

Trả về một list các node được kết nối với node hiện tại
'''
#nhận các nút liền kề
def getAdjNode(node):
    listNode = []
    emptyPos = get_pos(node.current_node, 0)

    for dir in DIRECTIONS.keys():
        newPos = (emptyPos[0] + DIRECTIONS[dir][0], emptyPos[1] + DIRECTIONS[dir][1])
        if 0 <= newPos[0] < len(node.current_node) and 0 <= newPos[1] < len(node.current_node[0]):
            newState = deepcopy(node.current_node)
            newState[emptyPos[0]][emptyPos[1]] = node.current_node[newPos[0]][newPos[1]]
            newState[newPos[0]][newPos[1]] = 0
            # listNode += [Node(newState, node.current_node, node.g + 1, Distance(newState), dir)]
            listNode.append(Node(newState, node.current_node, node.g + 1, Distance(newState), dir))
    return listNode


'''
Hàm lặp qua openSet.values() và trả về nút tốt nhất, được xác định bởi giá trị f() của nó.
Biến FirstIter theo dõi xem đây có phải là lần lặp đầu tiên thông qua danh sách hay không.
Code lặp lại thông qua danh sách openSet.values() và kiểm tra từng nút so với bestF 
để xem liệu nó có nên được trả về làm nút tốt nhất hay không.

sau đó kiểm tra xem có bất kỳ nút nào nữa trong openSet.values () hay không và trả về chúng nếu có.
'''
#lấy nút tốt nhất trong các nút có sẵn
def getBestNode(openSet):
    firstIter = True

    for node in openSet.values():
        if firstIter or node.f() < bestF:
            firstIter = False
            bestNode = node
            bestF = bestNode.f()
    return bestNode



'''
Hàm này tạo đường đi cho ô trống để đạt được trạng thái đích.

Tạo danh sách các nút trong closedSet
Bắt đầu lặp lại trên mỗi nút và đối với mỗi lần lặp, nó sẽ tạo ra một nhánh mới 
với nút hiện tại là nút gốc của nó.
Mỗi lần lặp, nó sẽ thêm vào 'branch' một đối tượng có 2 khóa: 'dir' và 'node'
'dir' là hướng mà nó đang trỏ vào và 'nút' là nút hiện tại của nó.
Sau đó đảo ngược thứ tự của danh sách này để khi nó trở về từ vòng lặp của nó, không còn dữ liệu nào còn lại trong nhánh.
 
Trả về branch là đường đi của bài toán
'''
#Hàm này tạo ra đường đi ngắn nhất
def buildPath(closedSet):
    node = closedSet[str(END)]
    branch = list()

    while node.dir:
        branch.append({
            'dir': node.dir,
            'node': node.current_node
        })
        node = closedSet[str(node.previous_node)]
    branch.append({
        'dir': '',
        'node': node.current_node
    })
    branch.reverse()
    return branch


'''
Giải thuật A*
Hàm sẽ tạo 2 tự điển là 'open_set' và 'closed_set'
Nó lặp qua từng nút trong tập 'open_set', tìm nút tốt nhất và thêm nó vào 'closed_set'.

Sau đó, nó quay trở lại bước 1 cho đến khi tất cả các nút đã được thêm vào hoặc không 
còn chỗ cho các nút trong các tập mở hoặc đóng.

Trước tiên kiểm tra xem có bất kỳ nút liền kề nào của nút hiện tại đã có trong danh sách
riêng của nó hay không (open_set)

Nó tiếp tục với nút liền kề tiếp theo trong danh sách của nó (adj_node) thay vì thêm 
nút mới này vào danh sách của riêng nó

Tạo một tập hợp các nút và đặt nút hiện tại thành nút tốt nhất trong tập hợp (adj_node)

Sau đó lặp lại qua tất cả các nút trong tập hợp đó, kiểm tra xem chúng đã có trong 'open_set' hay 'closed_set' không.

Nếu không, nó sẽ thêm chúng vào open_set và đóng closed_set
'''
#hàm chính (thuật toán A*)
def main(puzzle):
    open_set = {str(puzzle): Node(puzzle, puzzle, 0, Distance(puzzle), "")}
    closed_set = {}

    while True:
        test_node = getBestNode(open_set)
        closed_set[str(test_node.current_node)] = test_node

        if test_node.current_node == END:
            return buildPath(closed_set)

        adj_node = getAdjNode(test_node)
        for node in adj_node:
            if str(node.current_node) in closed_set.keys() or str(node.current_node) in open_set.keys() and open_set[str(node.current_node)].f() < node.f():
                continue
            open_set[str(node.current_node)] = node

        del open_set[str(test_node.current_node)]

'''
Đây là một câu lệnh có điều kiện kiểm tra chương trình để chạy
'''
if __name__ == '__main__':
    #it is start matrix
    br = main([[1,2,3,4],[5,6,0,8],[9,10,11,12],[13,14,15,7]])

    print('total steps : ', len(br) - 1)
    print()
    print(dash + dash + right_junction, "INPUT", left_junction + dash + dash)
    for b in br:
        if b['dir'] != '':
            letter = ''
            if b['dir'] == 'U':
                letter = 'UP'
            elif b['dir'] == 'R':
                letter = "RIGHT"
            elif b['dir'] == 'L':
                letter = 'LEFT'
            elif b['dir'] == 'D':
                letter = 'DOWN'
            print(dash + dash + right_junction, letter, left_junction + dash + dash)
        print_puzzle(b['node'])
        print()

    print(dash + dash + right_junction, 'ABOVE IS THE OUTPUT', left_junction + dash + dash)

total steps :  11

──┤ INPUT ├──
│ 1 │ 2 │ 3 │ 4 │
│ 5 │ 6 │   │ 8 │
│ 9 │ 10 │ 11 │ 12 │
│ 13 │ 14 │ 15 │ 7 │

──┤ RIGHT ├──
│ 1 │ 2 │ 3 │ 4 │
│ 5 │ 6 │ 8 │   │
│ 9 │ 10 │ 11 │ 12 │
│ 13 │ 14 │ 15 │ 7 │

──┤ DOWN ├──
│ 1 │ 2 │ 3 │ 4 │
│ 5 │ 6 │ 8 │ 12 │
│ 9 │ 10 │ 11 │   │
│ 13 │ 14 │ 15 │ 7 │

──┤ DOWN ├──
│ 1 │ 2 │ 3 │ 4 │
│ 5 │ 6 │ 8 │ 12 │
│ 9 │ 10 │ 11 │ 7 │
│ 13 │ 14 │ 15 │   │

──┤ LEFT ├──
│ 1 │ 2 │ 3 │ 4 │
│ 5 │ 6 │ 8 │ 12 │
│ 9 │ 10 │ 11 │ 7 │
│ 13 │ 14 │   │ 15 │

──┤ UP ├──
│ 1 │ 2 │ 3 │ 4 │
│ 5 │ 6 │ 8 │ 12 │
│ 9 │ 10 │   │ 7 │
│ 13 │ 14 │ 11 │ 15 │

──┤ RIGHT ├──
│ 1 │ 2 │ 3 │ 4 │
│ 5 │ 6 │ 8 │ 12 │
│ 9 │ 10 │ 7 │   │
│ 13 │ 14 │ 11 │ 15 │

──┤ UP ├──
│ 1 │ 2 │ 3 │ 4 │
│ 5 │ 6 │ 8 │   │
│ 9 │ 10 │ 7 │ 12 │
│ 13 │ 14 │ 11 │ 15 │

──┤ LEFT ├──
│ 1 │ 2 │ 3 │ 4 │
│ 5 │ 6 │   │ 8 │
│ 9 │ 10 │ 7 │ 12 │
│ 13 │ 14 │ 11 │ 15 │

──┤ DOWN ├──
│ 1 │ 2 │ 3 │ 4 │
│ 5 │ 6 │ 7 │ 8 │
│ 9 │ 10 │   │ 12 │
│ 13 │ 14 │ 11 │ 15 │

──┤ DOWN ├──
│ 1 │ 2 │ 3 │ 4 │
│ 5 │ 6 │ 7 │ 8 │
│ 9 │ 10 │ 11